In [2]:
import json
f = open('json_for_Muhammad.json')
data = json.load(f)


In [3]:
# importing the required libraries
import pandas as pd
import re
import ast
from nltk.tokenize import word_tokenize
import nltk
from nltk.tag import pos_tag
from nltk import RegexpParser
from nltk import Tree
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\4a\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
#  we are using 2 different POS taggers, 1 is of flair and the other one is of NLTK. The flair POS is more accurate
# but takes time to process while nltk is quick but can give less accurate results. Mainly we are using NLTK but
# at some point nltk was not giving good results so we used flair at that spot

# this takes a sentence and returns its POS tags using nltk
def preprocess1(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent



In [5]:
# the main processing is happing here. The parameter is the data the starting recipe and the ending recipe
def process(data,starting_recipe=0,ending_recipe=2):
# creating a empty variable to save final output
    df = None
# we have to manage 3 things from it now the food its quantity and its unit. So we creating 3 list to store 
# all of them in parallel
    des,quantity,unit = [],[],[]
    
# these are the words that we don't want in the food names. you can thing of them as stop words that we want to 
# remove and have no importance
    unknow_words = ['chopped',"slices","pinch","large","medium","small","each","cooked","uncooked","cooled","pot", "pouch", "packed" ,"package", "regular", "roughly" ,"slice", "sliced", "strips"]
# all the units that we can have in our recipes
    units = [' ml ', ' cup ',' can ',' cans ', ' cups '," liter "," liters ", " cm "," in ",' g ', ' oz ',' lb ', ' kg ',' kgs ', ' cm ', ' tbsp ', " ounces "," lbs ",' tsp ', ' l '," ea "]

# in case we are left with some units which our algo didn't successfully remove we can at the end remove using this
# list
    units_remove = ['%','/','.','-','_',']','*','-/', 'c.', 'à', 'thé','hot','pkg','half','and','cubed','cube', '/-inch', "in","cm",',packages', 'pitted', 'plain' ,'thin' ,'-inch' ,'inch' ,'cube', 'cubes','thinly','tiny','finely','[','ml', 'cup',"cups",'can','cans', "liter","-in","liters", 'g', 'oz', 'kg', 'cm','lb','-lb' ,'tbsp', 'kgs','tsp', 'l',"ea","ounces","lbs","peeled","roasted"]

#     we will now loop over all the data we have. we are using data["data"] as its the format of how our data is 
# stored in json
    for result in data['data'][starting_recipe:ending_recipe]:
    #     print(result["title"])
    
#     after excessing every recipe we need to extract its ingredients and loop over it to process them one by one
# in d we are storing in a form of list all the ingredients required for 1 recipe
        d= result[u'ingredients'][u'ungrouped'][u'list']
    
# we will loop over the ingredients now
        for a in d:
#         extracting the description of ingredient
            m= a['description']
#     removing all the dots inside the description
            m = re.sub("[.]"," ",m)
#     tokenizing and then rejoining. this is done to have space before and after every word so we can better process
            word_tokens = word_tokenize(m) 
            m  = (" ").join(word_tokens)
            
# We will split the description based on the ,
# we only need the part before , but the pattern breaks at only 2 places 
#     1- when we have NN,NN and when we have VBD,VBD eg boneless,chicken and cooled,cooked farro
            split_string1 = m.split(',')

# we are using POS tags and regexp together to catch any required pattern and if the pattern is detected keep 
# both sides of split

# defining the patterns
            done = 0
            if "," in m:
                patterns1 = """P: {<NN><,><NN>}
                                {<VBD><,><VBD>}
                                {<JJ><,><JJ>}    
                                {<VBN><,><VBN>}"""  # last one new change if issues due to 4 small roasted , peeled fresh beets , julienned

#                 2 small green , red and/or yellow bell peppers , thinly sliced
                
                #     giving regexp the patterns
                PChunker1 = RegexpParser(patterns1)
#     preprocess1() applied to function to get POS tags and then given to regexp to check it.
                output_pattern1 = PChunker1.parse(preprocess1(m))
#         the loop is check whether the pattern was found or not if found we will keep both side of split
                for child in output_pattern1:
                    if isinstance(child, Tree):              
                        if child.label() == 'P':

                            if (child[0][1] == "VBD") and len(split_string1)<3:

                                done=1
                                substring2 = split_string1[0] + "," +split_string1[1]
                            elif child[0][1] == "NN" or child[0][1] == "VBN" or child[0][1] == "JJ":
                                done=1
                                substring2 = split_string1[0] + "," +split_string1[1]

# if no pattern was found then we only need the first side and we do that here
            if done == 0:
                substring2 = split_string1[0]         
# spliting based on or. we don't need both side of or we only need the first side of or
# the pattern breaks only on JJ or JJ where we have to take the 2nd side of or eg dark or light break
            substring2 = re.sub("and/or","or",substring2)
            substring2 = re.sub("or/and","or",substring2)
            split_string = substring2.split(' or ')
            substring1 = ""
# we are doing same again making pattern giving it to regexp, getting pos for sentences and then checking if it
# exists or not
            done = 0
#     
            patterns = """P: {<JJ><CC><JJ>}
                             {<VBD><CC><VB>}
                             {<NN><CC><JJ>}
                             {<VBD><CC><JJ>}
                             {<CD><CC><CD>}""" #9-inch unbaked or frozen deep-dish pie crust *
            PChunker = RegexpParser(patterns)
            if " or " in substring2:
                found_pat = PChunker.parse(preprocess1(substring2))
#             else: 
#                 found_pat = PChunker.parse(preprocess1(substring2))

                for child in found_pat:
                    if isinstance(child, Tree):               
                        if child.label() == 'P':
                            if child[1][0] == "or":
                                done=1
                                substring1 = split_string[1]

            if done ==0:
                substring1 = split_string[0]

# we are lowering the whole string. thats due to the reason that we where having units in mixed cases like
# mL ml, ML and else so we need to lower it down all
            substring1 = substring1.lower()
# here we are spliting based on the units. we are giving priority to the unit which is coming after in description
# eg 1 g (20 ML) water so here ML will be prefered unit. We are storing that unit as well for later use
# after finding the unit we have to remove every thing before it as it was a pattern in all the descriptions
            index = 0
            u = "x"
            for x in units:
                finding = max(substring1.find(x),0)
                if finding > index:
                  index =finding
                  u = x

            if index !=0:
                index = index + len(u.strip())+1

#            this portion remove words inside bracket eg (optional)
#             substring3 = substring1[index:]
            substring3 = substring1
            substring3 = re.sub("\(.*?\)","",substring3)
            
# this portion clean the string by removing the brackets and its corresponding text
            index = [i for i in range(len(substring3)) if substring3.startswith("(", i)]

            if len(index) == 0:
                pass
            else:
                index = index[0]
                substring3 = substring3[:index]  


            index = [i for i in range(len(substring3)) if substring3.startswith(")", i)]

            if len(index) == 0:
                pass
            else:
                index = index[-1]
                substring3 = substring3[index+1:]      
# in the final name we don't have any kind of numbers. This portion is to detect any remaining no inside the
# description and remove them 
            x = re.findall("\d", substring3)

#             try:
#                 substring4 = substring3.split(x[-1])
#                 substring4 = substring4[-1]
#             except:
#                 substring4 = substring3
            for aaa in set(x):
                substring3 = re.sub(aaa,"",substring3)
            substring4 = substring3
# removing the stop words which we have defined above
            word_tokens = word_tokenize(substring4) 
            filtered_sentence = [w for w in word_tokens if not w in unknow_words+units_remove] 
            substring4  = (" ").join(filtered_sentence)



# removing extra spaces percentage signs and commas which are coming in starting or ending of strings
            substring4 = substring4.strip(",")
            substring4 = substring4.strip()
            
            if  "boneless" ==  substring4:
                print(m)
                print(substring1)
                print(substring2)
                print(substring3)
                print(substring4)
#             print(substring4)


#     adding data to all three lists and creating a dataframe out of it
            des.append(substring4)
            quantity.append(a['quantity'])
            unit.append(u)
            df =pd.DataFrame(zip(des,quantity,unit),columns=["food","quantity","unit"])

    return df

In [6]:
# running the function with data starting and ending index
start = 2
end = 23
start = max(0,start)
end = min(end,len(data))
df = process(data,0,455)

In [7]:
df.head()

,food,quantity,unit
0,knorr â® zaâ€™atar seasoning blend,60.00,ml
1,rapeseed oil,0.25,cup
2,head cauliflower,1.00,x
3,hellmann'sâ® real mayonnaise,60.00,ml
4,eggplant,1.00,x


In [8]:
from textblob import Word
df1 = df['food'].apply(lambda w: Word(w).singularize())

In [12]:
# in this portion we read all our categories file and store the data in form of list
bakery = []
with open("Bakery.txt") as f:
    bakery = [re.sub("\n","",a).strip() for a in f.readlines()]

dairy = []
with open("Dairy & Eggs.txt") as f:
     dairy = [re.sub("\n","",a).strip() for a in f.readlines()]

drinks = []
with open("Drinks.txt") as f:
    drinks = [re.sub("\n","",a).strip() for a in f.readlines()]
fruit = []
with open("Fruit & Vegetables.txt") as f:
    fruit = [re.sub("\n","",a).strip() for a in f.readlines()]

meat = []
with open("Meat & Fish.txt") as f:
    meat = [re.sub("\n","",a).strip() for a in f.readlines()]

pantry = []
with open("Pentry.txt") as f:
    pantry = [re.sub("\n","",a).strip() for a in f.readlines()]

In [13]:
# function which will determine which ingredient is of which category. It will match one by one with the list and
# as it will find a results it will return that category
def name_cat(name):
  check_bakery,check_pantry,check_meat,check_fruit,check_drink,check_dairy = False,False,False,False,False,False
  for a in fruit:
    if a in name and len(a)>0:
      return 0

  for a in dairy:
    if a in name and len(a)>0:
      return 1
  
  for a in bakery:
    if a in name and len(a)>0:
      return 2
  
  for a in drinks:
    if a in name and len(a)>0:
      return 3
  
  for a in meat:
    if a in name and len(a)>0:
      return 4
  
  for a in pantry:
    if a in name and len(a)>0:
      return 5

  return 6

In [14]:
# getting the total based on the food name and its unit
df1 = df.groupby(["food","unit"])

In [15]:
# looping our the grouped dataframe to extract the required info and get its category check and then store that 
# to a list
json_pantry = []
json_dairy = []
json_meat = []
json_bakery = []
json_vegetable = []
json_drink = []
json_others = []
for name,group in df1:
    new = {}
    new["ingredient"]=name[0]
    new["unit"]=name[1]
    new["quantity"]=group["quantity"].sum()
    cat = name_cat(name[0]) 
    if cat == 0:
        json_vegetable.append(new)
    elif cat == 1:
        json_dairy.append(new)
    elif cat == 2:
        json_bakery.append(new)
    elif cat == 3:
        json_drink.append(new)
    elif cat == 4:
        json_meat.append(new)
    elif cat == 5:
        json_pantry.append(new)
    elif cat == 6:
        json_others.append(new)
        

In [16]:
df1.head()

,food,quantity,unit
0,knorr â® zaâ€™atar seasoning blend,60.00,ml
1,rapeseed oil,0.25,cup
2,head cauliflower,1.00,x
3,hellmann'sâ® real mayonnaise,60.00,ml
4,eggplant,1.00,x
...,...,...,...
449,fresh pumpkin,225.00,g
451,halved cherry tomatoes,125.00,ml
452,sour dough bread,4.00,x
453,baby arugula,375.00,ml
